In [1]:
import numpy as np
import time
import scipy.spatial.distance as dis

データの用意

In [2]:
entries = np.random.rand(10000, 4096)
query = np.random.rand(4096)

for文で計算した場合

In [3]:
%%timeit
distance = []
for i in range(10000):
    distance.append(dis.cosine(entries[i], query))

1 loop, best of 3: 740 ms per loop


for文　内包表現

In [4]:
%%timeit
distance = [dis.cosine(entries[i], query) for i in range(10000)]

1 loop, best of 3: 731 ms per loop


In [5]:
entries = np.random.rand(10000, 4096)
query = np.random.rand(4096)

コサイン類似度の計算

In [6]:
%%timeit
distance = 1.0 - entries.dot(query) / (np.linalg.norm(entries, axis=1) * np.linalg.norm(query))

10 loops, best of 3: 90.7 ms per loop


In [7]:
entries = entries / np.linalg.norm(entries, axis=1).reshape(-1, 1)

予めentriesを正規化しておいた上で，距離のようなものを計算
（順番を知りたいだけなので1.0で引く必要が無い）

In [8]:
%%timeit
distance_sic = - entries.dot(query)

100 loops, best of 3: 15.2 ms per loop


GPUで計算する

In [13]:
import cupy as cp
# cupyにはcp.linalg.normがない
# またcupyはGPUのメモリ依存なのでサイズが大きいと計算できないかも
entries = cp.array(entries)
query = cp.array(query)

CUDARuntimeError: cudaErrorNoDevice: no CUDA-capable device is detected

In [18]:
%%timeit
distance_sic = - entries.dot(query)

10 loops, best of 3: 162 ms per loop


**つまりGPUの方が計算が遅い**